In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point
from datetime import datetime
import os
import seaborn as sns
import metpy.calc as mpcalc
import re
import calendar
from matplotlib.offsetbox import AnchoredText
sns.set()

In [ ]:
data_u = xr.open_dataset("uwnd_79_20.nc")
data_u.variables['latitude']   #array 1D
data_u.variables['longitude']  #array 1D

In [ ]:
data_u


In [ ]:
data_v = xr.open_dataset("vwnd_79_20.nc")
data_v

In [ ]:
def set_data(tempo, level):
    
    datau = xr.open_dataset("uwnd_79_20.nc")
    u = datau.u[:,0,level,:,:]
    
    datav = xr.open_dataset("vwnd_79_20.nc")
    v = datav.v[:,0,level,:,:]
    
    time_slice = slice(tempo[0],tempo[1])
    
    Umedia_mes = u.sel(time=time_slice).groupby('time.month').mean()
    Vmedia_mes = v.sel(time=time_slice).groupby('time.month').mean()
    
    lons = Umedia_mes.longitude.values    #1D
    lats = Vmedia_mes.latitude.values     #1D
    
    
    return lons, lats, Umedia_mes, Vmedia_mes

In [ ]:
def figura(tempo1, tempo2, level, mes):
    
    fig, ax = plt.subplots(nrows=1,ncols=2,subplot_kw=dict(projection = ccrs.PlateCarree()), figsize=(32,26))
    if level == 0:
        fig.suptitle('Comparativo\nCampo de ventos 700 hPA', fontsize=34, weight='bold', y = 0.75, x = 0.505)
    else:
        fig.suptitle('Comparativo\nCampo de ventos 850 hPA', fontsize=34, weight='bold', y = 0.75, x = 0.505)
    #mes = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    plott(ax[0], mes, tempo1, level)
    plott2(ax[1], mes, tempo2, level)
   

In [ ]:
def plott(ax, mes, tempo1, level):
    
    lons, lats, Umedia_mes, Vmedia_mes = set_data(tempo1 , level)
    
    Uvel, lonu = add_cyclic_point(Umedia_mes.sel(month=mes), coord=lons)  #interpolando o grafico/ corrigindo a descontinuidade do meridiano
    Vvel, lonv = add_cyclic_point(Vmedia_mes.sel(month=mes), coord=lons)
    
    
    #lons                                    1D
    #lats                                    1D
    #Uvel = Umedia_mes.sel(month=mes)        #2D
    #Vvel = Vmedia_mes.sel(month=mes)        #2D
    
    #clevs = np.arange(0,14,0.5)
    #cf = ax.contourf(coordenadas[0],coordenadas[1], media_mespos.sel(month = mes), clevs, cmap = 'jet_r')
    
    magnitude = (Uvel ** 2 + Vvel ** 2) ** 0.5
    
    norm = mpl.colors.Normalize(vmin=0, vmax=30)
    
    sp = ax.streamplot(lonu, lats, Uvel, Vvel,
                   linewidth = 1,
                   arrowsize = 1,
                   density = 5,
                   color = magnitude,
                   cmap='jet_r',
                   norm=norm,
                   transform=ccrs.PlateCarree())
    
    #cbar = plt.colorbar(cf, orientation='horizontal', pad=0.05 , ax=ax, shrink = 1.0, aspect=40)
    #cbar.ax.tick_params(labelsize=15)
    #cbar.set_label("m/s", fontsize = 17, y = 0.3)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}
    
    ax.coastlines('50m')
    ax.set_extent([-70, 20, -50, 10])
    ax.set_title(calendar.month_abbr[mes], fontdict={'fontsize': 30}, loc = 'right')
    ax.set_title('Climatologia de 1979 até 2010', fontdict={'fontsize': 30}, loc = 'left')

In [ ]:
def plott2(ax, mes, tempo2, level):
    
    lons, lats, Umedia_mes, Vmedia_mes = set_data(tempo2 , level)
    
    Uvel, lonu = add_cyclic_point(Umedia_mes.sel(month=mes), coord=lons)  #interpolando o grafico/ corrigindo a descontinuidade do meridiano
    Vvel, lonv = add_cyclic_point(Vmedia_mes.sel(month=mes), coord=lons)
    
    
    #lons                                    1D
    #lats                                    1D
    #Uvel = Umedia_mes.sel(month=mes)        #2D
    #Vvel = Vmedia_mes.sel(month=mes)        #2D
    
    #clevs = np.arange(0,14,0.5)
    #cf = ax.contourf(coordenadas[0],coordenadas[1], media_mespos.sel(month = mes), clevs, cmap = 'jet_r')
    
    magnitude = (Uvel ** 2 + Vvel ** 2) ** 0.5
    
    norm = mpl.colors.Normalize(vmin=0, vmax=30)
    
    sp = ax.streamplot(lonu, lats, Uvel, Vvel,
                   linewidth = 1,
                   arrowsize = 1,
                   density = 5,
                   color = magnitude,
                   cmap='jet_r',
                   norm=norm,
                   transform=ccrs.PlateCarree())
    
    #cbar = plt.colorbar(cf, orientation='horizontal', pad=0.05 , ax=ax, shrink = 1.0, aspect=40)
    #cbar.ax.tick_params(labelsize=15)
    #cbar.set_label("m/s", fontsize = 17, y = 0.3)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}
    
    ax.coastlines('50m')
    ax.set_extent([-70, 20, -50, 10])
    ax.set_title(calendar.month_abbr[mes], fontdict={'fontsize': 30}, loc = 'right')
    ax.set_title('2018 até 2019', fontdict={'fontsize': 30}, loc = 'left')

In [ ]:
tempo1 = ['1979-01-01','2010-12-01']
tempo2 = ['2018-01-01', '2019-12-01']

# 0 = '700' hPA
# 1 = '850' hPA
#figura(tempo1, tempo2, level, mes escolhido)
    #level: 0 = '700' hPA   ou   1 = '850' hPA  
    #mes escolhido = [1 até 12]

In [ ]:
figura(tempo1, tempo2, 0, 1)
plt.savefig("uwnd_vwnd.png")

In [ ]:
figura(tempo1,tempo2, 0, 5)